In [101]:
from riotwatcher import RiotWatcher
import pandas as pd
from converters.data2frames import game_to_dataframe as g2df
from converters.data2frames import game_teams_to_dataframe as gt2df
from converters.data2frames import champs_to_dataframe as ch2df
from converters.data2files import read_json
from config.constants import API_KEY, SLO_GAMES_DIR, SLO_DATASETS_DIR, SLO_DATASET_CSV, SLO_MATCHES_FILE_PATH, SLO_DATA_DTYPES
from config.constants import DATA_DRAGON_URL, DD_RUNES_REFORGED, DD_LANGUAGE, DD_RUNES_REFORGED, STATIC_DATA_DIR
from config.constants import SLO_MATCHES_DATA_P_COLS, SLO_CUSTOM_POSITIONS
import numpy as np


In [67]:
df2 = pd.read_csv('df2.csv', index_col=0)